In [1]:
import tensorflow as tf
print(tf.__version__)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

2.4.1


SystemError: GPU device not found

In [2]:
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
!tar zxf ldcc-20140209.tar.gz > /dev/null

--2021-03-27 19:44:30--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
www.rondhuit.com (www.rondhuit.com) をDNSに問いあわせています... 59.106.19.174
www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 8855190 (8.4M) [application/x-gzip]
`ldcc-20140209.tar.gz.1' に保存中

ldcc-20140209.tar.g 100%[===================>]   8.44M   114KB/s 時間 72s        

2021-03-27 19:45:44 (119 KB/s) - `ldcc-20140209.tar.gz.1' へ保存完了 [8855190/8855190]



In [2]:
import glob
import re

import MeCab
import pandas as pd

# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
categories = [
    "sports-watch", "topic-news", "dokujo-tsushin", "peachy",
    "movie-enter", "kaden-channel", "livedoor-homme", "smax",
    "it-life-hack",
]

In [4]:
docs = []
for category in categories:
    for f in glob.glob(f"./text/{category}/{category}*.txt"):
        # 1ファイルごとに処理
        with open(f, "r", encoding="utf-8") as fin:
            # nextで1行取得する(__next__を呼ぶ)
            url = next(fin).strip()
            date = next(fin).strip()
            title = next(fin).strip()
            body = "\n".join([line.strip() for line in fin if line.strip()])
      
        docs.append((category, url, date, title, body))

In [5]:
df = pd.DataFrame(
        docs,
        columns=["category", "url", "date", "title", "body"],
        dtype="category"
)

df["date"] = pd.to_datetime(df["date"])

In [6]:
le = LabelEncoder()

df = df.assign(
    label=lambda df: pd.Series(le.fit_transform(df.category))
)

In [7]:
idx = df.index.values

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
idx_train, idx_val = train_test_split(idx, random_state=123)
df_train = df.loc[idx_train, ['body', 'label']]
df_val = df.loc[idx_val, ['body', 'label']]

In [10]:
import tensorflow as tf

data_no_wakati = {
    "train": tf.data.Dataset.from_tensor_slices({
        'sentence1': df_train['body'].tolist(),
        'sentence2': ['', ] * len(df_train),
        'label': df_train['label'].tolist()
    }),
    "validation": tf.data.Dataset.from_tensor_slices({
        'sentence1': df_val['body'].tolist(),
        'sentence2': ['', ] * len(df_val),
        'label': df_val['label'].tolist()
    })
}

In [11]:
import tensorflow as tf
from transformers import BertJapaneseTokenizer

In [12]:
def tokenize_map_fn(tokenizer, max_length=100):
    """map function for pretrained tokenizer"""
    def _tokenize(text_a, text_b, label):
        inputs = tokenizer.encode_plus(
            text_a.numpy().decode('utf-8'),
            text_b.numpy().decode('utf-8'),
            add_special_tokens=True,
            max_length=max_length,
        )
        input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        attention_mask = [1] * len(input_ids)
        return input_ids, token_type_ids, attention_mask, label
    
    def _map_fn(data):
        text_a = data['sentence1']
        text_b = data['sentence2']
        label = data['label']
        out = tf.py_function(_tokenize, inp=[text_a, text_b, label], Tout=(tf.int32, tf.int32, tf.int32, tf.int32))
        return (
            {"input_ids": out[0], "token_type_ids": out[1], "attention_mask": out[2]},
            out[3]
        )
    return _map_fn

In [13]:
# Load dataset, tokenizer, model from pretrained vocabulary

def load_dataset(data, tokenizer, max_length=128, train_batch=8, val_batch=32):
    # Prepare dataset for BERT as a tf.data.Dataset instance
    train_dataset = data['train'].map(tokenize_map_fn(tokenizer, max_length=max_length))
    valid_dataset = data['validation'].map(tokenize_map_fn(tokenizer, max_length=max_length))
    train_dataset = train_dataset.shuffle(100).padded_batch(train_batch, padded_shapes=({'input_ids': max_length, 'token_type_ids': max_length, 'attention_mask': max_length}, []), drop_remainder=True)
    valid_dataset = valid_dataset.padded_batch(val_batch, padded_shapes=({'input_ids': max_length, 'token_type_ids': max_length, 'attention_mask': max_length}, []), drop_remainder=True)
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    valid_dataset = valid_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return train_dataset, valid_dataset

In [14]:
import ipywidgets

data = {
    "train": tf.data.Dataset.from_tensor_slices({
        'sentence1': df_train['body'].tolist(),
        'sentence2': ['', ] * len(df_train),
        'label': df_train['label'].tolist()
    }),
    "validation": tf.data.Dataset.from_tensor_slices({
        'sentence1': df_val['body'].tolist(),
        'sentence2': ['', ] * len(df_val),
        'label': df_val['label'].tolist()
    })
}


tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

#OSError: Model name 'bert-base-japanese' was not found in tokenizers model name list 
#(bert-base-japanese, bert-base-japanese-whole-word-masking, bert-base-japanese-char, 
#bert-base-japanese-char-whole-word-masking). We assumed 'bert-base-japanese' was a path, a model identifier, 
#or url to a directory containing vocabulary files named ['vocab.txt'] but couldn't find such vocabulary files at this path or url.

#https://github.com/huggingface/transformers/issues/4060
#!pip install git+git://github.com/huggingface/transformers.git

#train_dataset, valid_dataset = load_dataset(data)
train_dataset, valid_dataset = load_dataset(data, tokenizer)

#https://www.haya-programming.com/entry/2018/10/05/233418
#TypeError: load_dataset() missing 1 required positional argument: 'tokenizer'


for data in train_dataset.take(1):
    print(data)

#!pip install pip3 install --upgrade tensorflow 
#!pip install ipadic
#!pip install ipywidgets
#!pip install fugashi

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


({'input_ids': <tf.Tensor: shape=(8, 128), dtype=int32, numpy=
array([[    2, 19300, 14230, ...,    51,     3,     3],
       [    2,  1758,     5, ...,  1037,     3,     3],
       [    2,  6338,     6, ..., 14071,     3,     3],
       ...,
       [    2,  2292,    12, ...,  5858,     3,     3],
       [    2,    36, 20617, ...,     5,     3,     3],
       [    2,  1789,  3057, ..., 16422,     3,     3]])>, 'token_type_ids': <tf.Tensor: shape=(8, 128), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])>, 'attention_mask': <tf.Tensor: shape=(8, 128), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])>}, <tf.Tensor: shape=(8,), dtype=int32, nu

In [15]:
from transformers import TFBertModel

from tensorflow import keras
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout

In [16]:
bert = TFBertModel.from_pretrained('cl-tohoku/bert-base-japanese')

Some layers from the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at cl-tohoku/bert-base-japanese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [17]:
bert.layers

In [18]:
def make_model(bert, num_classes, max_length, bert_frozen=True):
    # use only first layer and froze it
    bert.layers[0].trainable = not bert_frozen

    # input
    input_ids = Input(shape=(max_length, ), dtype='int32', name='input_ids')
    attention_mask = Input(shape=(max_length, ), dtype='int32', name='attention_mask')
    token_type_ids = Input(shape=(max_length, ), dtype='int32', name='token_type_ids')
    inputs = [input_ids, attention_mask, token_type_ids]

    # bert
    x = bert.layers[0](inputs)
    # x: sequence_output, pooled_output

    # only use pooled_output
    out = x[1]

    # fc layer(add layers for transfer learning)
    out = Dropout(0.25)(out)
    out = Dense(128, activation='relu')(out)
    out = Dropout(0.5)(out)
    out = Dense(num_classes, activation='softmax')(out)
    return Model(inputs=inputs, outputs=out)


In [19]:
max_length = 128
num_classes = 9
model = make_model(bert, num_classes, max_length)

pred = model.predict(next(iter(train_dataset))[0])
print(pred.shape)
print(pred[0])

(8, 9)
[0.05616339 0.09231102 0.12533331 0.12583362 0.16613422 0.11072103
 0.09673732 0.04189603 0.18487014]


In [20]:
# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule 
optimizer = optimizers.Adam()
loss = losses.SparseCategoricalCrossentropy()
metric = metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [21]:
sample = next(iter(train_dataset))
evl = model.evaluate(sample[0], sample[1], verbose=0)
print('evaluate acc by model: ', evl[1])

import numpy as np
pred = model.predict(sample[0])
labels_np = sample[1].numpy()
acc = np.sum((np.argmax(pred, axis=-1) == labels_np).astype(np.int64)) / labels_np.shape[0]
print('evaluate acc by hand: ', acc)

evaluate acc by model:  0.0
evaluate acc by hand:  0.0


In [22]:
# ---- bert-base-japanese ----
epochs = 7
max_length = 500
train_batch = 32
val_batch = 64
num_classes = 9

# Load dataset, tokenizer, model from pretrained vocabulary
#tokenizer = BertJapaneseTokenizer.from_pretrained('bert-base-japanese')
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')
train_dataset, valid_dataset = load_dataset(data_no_wakati, tokenizer, max_length=max_length, train_batch=train_batch, val_batch=val_batch)

# define fine-tuning model
bert = TFBertModel.from_pretrained('cl-tohoku/bert-base-japanese')
#bert = TFBertModel.from_pretrained('bert-base-japanese')
model = make_model(bert, num_classes, max_length)
optimizer = optimizers.Adam()
loss = losses.SparseCategoricalCrossentropy()
metric = metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# # Train and evaluate using tf.keras.Model.fit()
history = model.fit(train_dataset, epochs=epochs,
                    validation_data=valid_dataset)

Some layers from the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at cl-tohoku/bert-base-japanese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/7


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


  3/172 [..............................] - ETA: 1:01:50 - loss: 2.7798 - accuracy: 0.0955

KeyboardInterrupt: 